In [1]:
# Surprise 라이브러리 설치
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 18.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=1381417 sha256=b138cead3d4bc2e9b9e53b501a9247d928d4ec0bcfe0b8155f8385360212cb39
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
# Requirements
import os
import boto3
import sagemaker
import numpy as np
import pandas as pd
import sys
import joblib
from tqdm import tqdm
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate, train_test_split, KFold
from surprise import accuracy
from surprise import AlgoBase
from surprise.prediction_algorithms.predictions import Prediction
from surprise.trainset import Trainset
from surprise import PredictionImpossible
from sklearn.metrics.pairwise import cosine_similarity
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.tensorflow import TensorFlow
from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [26]:
# 데이터셋 로드
df = pd.read_csv('./data/restaurant_rating_dataset.csv')

In [55]:
# 상위 5000개 business와 상위 5000개의 user 데이터를 선택
top_businesses = df['business']
top_users = df['user'].value_counts().index[:10000]

In [56]:
# 'business'가 상위 15,000개 중 하나이고 'user'가 상위 10,000개 중 하나인 행들만을 유지
df = df[df['business'].isin(top_businesses) & df['user'].isin(top_users)]

In [58]:
# Load data
business_df = pd.read_csv('./data/restaurant_unique_business.csv')
user_df = pd.read_csv('./data/restaurant_unique_user.csv')
rest_df = pd.read_csv('./data/preprocessed-meta-California.csv')

/tmp/ipykernel_7692/3049391663.py:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  rest_df = pd.read_csv('./data/preprocessed-meta-California.csv')


In [62]:
# Merge recommend_df with business_series based on the business column
result_df = pd.merge(df, business_df, how='left', left_on='business', right_index=True)

In [64]:
result_df

,business_x,user,rating,index,business_y
247546,7332,10610,5,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c
247771,7332,120111,5,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c
247827,7332,15439,5,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c
247904,7332,15581,4,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c
247918,7332,222497,4,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c
...,...,...,...,...,...
29074905,80633,48862,5,80633,0x80dd3125a85f8ef3:0x13a546608196176c
29074918,80633,13085,5,80633,0x80dd3125a85f8ef3:0x13a546608196176c
29074962,80633,132256,4,80633,0x80dd3125a85f8ef3:0x13a546608196176c
29074980,80633,43306,5,80633,0x80dd3125a85f8ef3:0x13a546608196176c


In [65]:
# 'business' 컬럼을 인덱스로 설정
result_df.set_index('business_x', inplace=True)

# 인덱스를 열로 변환
result_df.reset_index(inplace=True)

# 'business' 컬럼의 데이터 타입을 문자열로 변환
result_df['business_x'] = result_df['business_x'].astype(str)

In [66]:
# Merge recommend_df and rest_df based on the common column
result_df_2 = pd.merge(result_df, rest_df, how='left', left_on='business_y', right_on='gmap_id')

In [67]:
result_df_2

,business_x,user,rating,index,business_y,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,7332,10610,5,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Water Grill,"Water Grill, 544 S Grand Ave, Los Angeles, CA ...",0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Downtown landmark presents a wide array of del...,34.049040,-118.254707,['Seafood restaurant'],4.4,1378,$$$,"[['Monday', '11:30AM–9PM'], ['Tuesday', '11:30...","{'Service options': ['Takeaway', 'Dine-in', 'D...",Open ⋅ Closes 9PM,"['0x80c2c7b14dfa1c77:0x292681801d633db0', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
1,7332,120111,5,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Water Grill,"Water Grill, 544 S Grand Ave, Los Angeles, CA ...",0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Downtown landmark presents a wide array of del...,34.049040,-118.254707,['Seafood restaurant'],4.4,1378,$$$,"[['Monday', '11:30AM–9PM'], ['Tuesday', '11:30...","{'Service options': ['Takeaway', 'Dine-in', 'D...",Open ⋅ Closes 9PM,"['0x80c2c7b14dfa1c77:0x292681801d633db0', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2,7332,15439,5,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Water Grill,"Water Grill, 544 S Grand Ave, Los Angeles, CA ...",0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Downtown landmark presents a wide array of del...,34.049040,-118.254707,['Seafood restaurant'],4.4,1378,$$$,"[['Monday', '11:30AM–9PM'], ['Tuesday', '11:30...","{'Service options': ['Takeaway', 'Dine-in', 'D...",Open ⋅ Closes 9PM,"['0x80c2c7b14dfa1c77:0x292681801d633db0', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
3,7332,15581,4,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Water Grill,"Water Grill, 544 S Grand Ave, Los Angeles, CA ...",0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Downtown landmark presents a wide array of del...,34.049040,-118.254707,['Seafood restaurant'],4.4,1378,$$$,"[['Monday', '11:30AM–9PM'], ['Tuesday', '11:30...","{'Service options': ['Takeaway', 'Dine-in', 'D...",Open ⋅ Closes 9PM,"['0x80c2c7b14dfa1c77:0x292681801d633db0', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
4,7332,222497,4,7332,0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Water Grill,"Water Grill, 544 S Grand Ave, Los Angeles, CA ...",0x80c2c7b4901ce7c7:0x2ad996c9a82b097c,Downtown landmark presents a wide array of del...,34.049040,-118.254707,['Seafood restaurant'],4.4,1378,$$$,"[['Monday', '11:30AM–9PM'], ['Tuesday', '11:30...","{'Service options': ['Takeaway', 'Dine-in', 'D...",Open ⋅ Closes 9PM,"['0x80c2c7b14dfa1c77:0x292681801d633db0', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111892,80633,48862,5,80633,0x80dd3125a85f8ef3:0x13a546608196176c,Parkers' Lighthouse,"Parkers' Lighthouse, 435 Shoreline Village Dri...",0x80dd3125a85f8ef3:0x13a546608196176c,"Mesquite-grilled seafood, sushi, steaks & a ro...",33.759861,-118.191797,"['Seafood restaurant', 'Bar', 'Diner', 'Lounge...",4.5,1880,$$$,"[['Wednesday', '11:30AM–3PM'], ['Thursday', '1...","{'Service options': ['Outdoor seating', 'Curbs...",NaN,"['0x80dd3125a8dd68e1:0xcadaa87a95de641f', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
111893,80633,13085,5,80633,0x80dd3125a85f8ef3:0x13a546608196176c,Parkers' Lighthouse,"Parkers' Lighthouse, 435 Shoreline Village Dri...",0x80dd3125a85f8ef3:0x13a546608196176c,"Mesquite-grilled seafood, sushi, steaks & a ro...",33.759861,-118.191797,"['Seafood restaurant', 'Bar', 'Diner', 'Lounge...",4.5,1880,$$$,"[['Wednesday', '11:30AM–3PM'], ['Thursday', '1...","{'Service options': ['Outdoor seating', 'Curbs...",NaN,"['0x80dd3125a8dd68e1:0xcadaa87a95de641f', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
111894,80633,132256,4,80633,0x80dd3125a85f8ef3:0x13a546608196176c,Parkers' Lighthouse,"Parkers' Lighthouse, 435 Shoreline Village Dri...",0x80dd3125a85f8ef3:0x13a546608196176c,"Mesquite-grilled seafood, sushi, steaks & a ro...",33.759

In [68]:
result_df_2.info()
result_df_2.to_csv('./data/top_rest_rating.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111897 entries, 0 to 111896
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   business_x        111897 non-null  object 
 1   user              111897 non-null  int64  
 2   rating            111897 non-null  int64  
 3   index             111897 non-null  int64  
 4   business_y        111897 non-null  object 
 5   name              111897 non-null  object 
 6   address           111897 non-null  object 
 7   gmap_id           111897 non-null  object 
 8   description       106900 non-null  object 
 9   latitude          111897 non-null  float64
 10  longitude         111897 non-null  float64
 11  category          111897 non-null  object 
 12  avg_rating        111897 non-null  float64
 13  num_of_reviews    111897 non-null  int64  
 14  price             101899 non-null  object 
 15  hours             108976 non-null  object 
 16  MISC              11

In [30]:
df.groupby('user').business.nunique().min()

2

In [31]:
def train_test_split(df, holdout_num):
    """
    Splits the data into training and testing sets.

    @param df The input dataframe
    @param holdout_num The number of items to hold out for testing for each user

    @return df_train training data
    @return df_test testing data
    """
    # Sort the data by 'user' and 'business'
    df = df.sort_values(['user', 'business'], ascending=True)

    # Select a subset of the data for testing
    df_test = df.groupby('user').head(holdout_num)

    # The remaining data is used for training
    df_train = df[~df.index.isin(df_test.index)]

    return df_train, df_test

In [32]:
df_train, df_test = train_test_split(df, 10)

In [33]:
def negative_sampling(user_ids, business_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label
    
    @param user_ids: list of user ids
    @param movie_ids: list of movie ids
    @param items: unique list of movie ids
    @param n_neg: number of negative labels to sample
    
    @return df_neg: negative sample dataframe
    
    """
    
    neg = []
    ui_pairs = zip(user_ids, business_ids)
    records = set(ui_pairs)
    
    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            # if the randomly sampled movie exists for that user
            j = np.random.choice(items)
            while(u, j) in records:
                # resample
                j = np.random.choice(items)
            neg.append([u, j, 0])
    # conver to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['user', 'business', 'rating'])
    
    return df_neg

In [34]:
# create negative samples for training set
neg_train = negative_sampling(
    user_ids=df_train.user.values, 
    business_ids=df_train.business.values,
    items=df.business.unique(),
    n_neg=5
)

In [35]:
print(f'created {neg_train.shape[0]:,} negative samples')

created 402,835 negative samples


In [36]:
df_train = df_train[['user', 'business']].assign(rating=1)
df_test = df_test[['user', 'business']].assign(rating=1)

df_train = pd.concat([df_train, neg_train], ignore_index=True)

In [37]:
def get_unique_count(df):
    """calculate unique user and business counts"""
    return df.user.nunique(), df.business.nunique()

In [38]:
# unique number of user and movie in the whole dataset
get_unique_count(df)

(2999, 2999)

In [39]:
print('training set shape', get_unique_count(df_train))
print('testing set shape', get_unique_count(df_test))

training set shape (2979, 2999)
testing set shape (2999, 1973)


In [40]:
# number of unique user and number of unique item/movie
n_user, n_item = get_unique_count(df_train)

print("number of unique users", n_user)
print("number of unique items", n_item)

number of unique users 2979
number of unique items 2999


In [41]:
# save the variable for the model training notebook
# -----
# read about `store` magic here: 
# https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html

%store n_user
%store n_item

Stored 'n_user' (int)
Stored 'n_item' (int)


In [42]:
# get current session region
session = boto3.session.Session()
region = session.region_name
print(f'currently in {region}')

currently in ap-northeast-2


In [43]:
# 처리된 데이터를 저장하기 위해서 기본 sagemaker s3 버킷을 사용합니다. 
# 여기서는 기본 버킷 이름이 다음의 코드가 나타냅니다. 
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
print(bucket_name)  
# bucket name format: "sagemaker-{region}-{aws_account_id}"
%store bucket_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker-ap-northeast-2-629515838455
Stored 'bucket_name' (str)


In [46]:
# use the default sagemaker s3 bucket to store processed data
# here we figure out what that default bucket name is 
sagemaker_session = sagemaker.Session()
bucket_name = 'gcu-sg02-002'
print(bucket_name)  
# bucket name format: "sagemaker-gacheon-{account 숫자}"
%store bucket_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
gcu-sg02-002
Stored 'bucket_name' (str)


In [47]:
# save data locally first
dest = 'data/s3'
train_path = os.path.join(dest, 'train.npy')
test_path = os.path.join(dest, 'test.npy')

!mkdir {dest}
np.save(train_path, df_train.values)
np.save(test_path, df_test.values)

# upload to S3 bucket (see the bucket name above)
sagemaker_session.upload_data(train_path, bucket=bucket_name, key_prefix='data')
sagemaker_session.upload_data(test_path, bucket=bucket_name, key_prefix='data')

mkdir: cannot create directory ‘data/s3’: File exists


S3UploadFailedError: Failed to upload data/s3/train.npy to gcu-sg02-002/data/train.npy: An error occurred (AccessDenied) when calling the CreateMultipartUpload operation: Access Denied

In [6]:
# pivot_table 메소드를 사용하여 rating matrix 생성
rating_matrix_df = filtered_df.pivot_table(values='rating', index='user', columns='business', fill_value=0)

# csr_matrix를 이용하여 sparse matrix로 변환
rating_matrix = csr_matrix(rating_matrix_df.values)

# 피어슨 상관계수를 이용해 유사도를 계산 
def pearson_similarity(matrix):
    matrix = matrix.toarray()  # Transpose matrix for user-based similarities
    similarity_matrix = 1 - pairwise_distances(matrix, metric="correlation", n_jobs=-1)  # Use multiple CPU cores for faster computation
    return similarity_matrix

# user 간의 피어슨 유사도 계산
user_similarity = pearson_similarity(rating_matrix)

# 피어슨 유사도를 유저들 간의 유사도를 나타내는 데이터프레임 형태로 변환
user_similarity_df = pd.DataFrame(user_similarity, index=rating_matrix_df.index, columns=rating_matrix_df.index)

%store rating_matrix_df
%store user_similarity_df

Stored 'rating_matrix_df' (DataFrame)
Stored 'user_similarity_df' (DataFrame)


In [7]:
# # 메모리 사용량 확인
# memory_usage = filtered_df.memory_usage(deep=True).sum()
# print("DataFrame의 메모리 사용량:", memory_usage / (1024**2), "MB")

In [8]:
# Surprise Reader 정의 (평점의 범위를 지정해줘야 함)
reader = Reader(rating_scale=(1, 5))

In [9]:
# 데이터를 Surprise 형식으로 로딩
data = Dataset.load_from_df(filtered_df[['user', 'business', 'rating']], reader)

In [10]:
# KFold를 사용한 cross-validation 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# KNNBasic 알고리즘 생성 (유사도 측정 방법 'cosine'를 사용, n_epochs를 지정)
model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True}, n_epochs=20)

# cross-validation을 통한 모델 성능 평가
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=kf, verbose=True)

# cross-validation 결과 출력
for measure in ['test_rmse', 'test_mae']:
    print(f"{measure}: {results[measure].mean()}")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9744  0.9772  0.9758  0.9720  0.9736  0.9746  0.0018  
MAE (testset)     0.7668  0.7670  0.7674  0.7643  0.7616  0.7654  0.0022  
Fit time          2.17    2.31    2.53    2.33    2.52    2.37    0.14    
Test time         3.62    3.24    3.78    3.64    4.05    3.67    0.26    
test_rmse: 0.9746070872223832
test_mae: 0.7654274623966686


In [11]:
# 가장 성능이 좋은 폴드의 모델을 선택
best_fold_idx = results['test_rmse'].argmin()
best_model = model

In [12]:
# 선택된 모델로 전체 데이터에 대해 훈련
trainset = data.build_full_trainset()
best_model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [21]:
print(rating_matrix_df)
print(user_similarity_df)

business  7185   7239   7330   7332   7341   7351   7377   7406   7421   \
user                                                                      
118           0      0      0      0      0      0      0      0      0   
128           0      0      0      0      0      0      0      0      0   
184           0      0      0      0      0      0      0      0      0   
199           0      4      0      0      0      0      0      0      0   
204           0      5      0      0      0      0      0      0      0   
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
1244522       0      0      0      0      0      0      0      0      0   
1248454       0      0      0      0      0      0      0      0      0   
1311498       0      0      0      0      0      0      0      0      0   
1362023       0      0      0      0      0      0      0      0      0   
1997603       0      0      0      0      0      0      0      0      0   

business  7422   ...  80

In [27]:
# 예시: 0번 유저에 대해 5개의 비즈니스에 대한 평점 예측
user_id_to_predict = 21788908987
items_to_predict = rating_matrix_df.columns[:5]  # 상위 5개의 비즈니스
for item_id in items_to_predict:
    prediction = best_model.predict(user_id_to_predict, item_id)
    print(f"Predicted rating for business {item_id}: {prediction.est}")

Predicted rating for business 7185: 4.225025169869379
Predicted rating for business 7239: 4.225025169869379
Predicted rating for business 7330: 4.225025169869379
Predicted rating for business 7332: 4.225025169869379
Predicted rating for business 7341: 4.225025169869379
